# Importation des packages necessaires à l'étude 

In [2]:
# Import des packages
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import scipy.stats as st
import os
import sys

sys.path.insert(0, str(Path(os.getcwd()).resolve().parent.parent))
from Churn_contrat_auto.src.utils.tools import (
    rename_field_categories,
    distrib_for_cat_by_target,
    distrib_for_cont_by_target,
    boxplot_by_target,
    plot_generic,
    cramers_v_between_all,
    cramers_v_with_target,
    graph_correlations,
    extract_corr_pairs,
)

#### Objectif : cibler les clients qui vont résilier leur contrat auto

# 1. Import des données

In [80]:
df = pd.read_csv("/Users/melaniedaddio/Desktop/Cours_M2/M2_S2/scoring/Churn_contrat_auto/data/raw/ResiliationContratAuto.txt", 
                 sep = '|',
                 encoding = 'latin1') # On rajoute l'encoding car ici nous ne sommes pas en utf8
df.head()

/var/folders/74/x3782kls1rv6qscx40xcj5rm0000gn/T/ipykernel_31527/20778580.py:1: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/melaniedaddio/Desktop/Cours_M2/M2_S2/scoring/Churn_contrat_auto/data/raw/ResiliationContratAuto.txt",


,NO_AFR,CDUSGAUT,IDECON,CD_AGT,CD_FML,CDPRGES,DTDBUCON,DTEFTMVT,MMJECHPP,MTPAATTC,...,RESIV,RESDI,CLIACTIF,U,CONTRAT,ETAT,MOTIFRSL,COEFCOMM,COEFPFLT,CRM
0,A00104001204707,611.0,205808992,A00104,R,4190A,14630,14812,121.0,1929.66,...,NaN,NaN,OUI,0.630638,ACTIF,CO,NaN,100,0.8500,50
1,A00104001270809,636.0,105939601,A00104,C,4190A,14701,14701,401.0,4106.44,...,NaN,NaN,OUI,0.012901,ACTIF,CO,NaN,100,0.7500,76
2,A00104001397301,611.0,306115473,A00104,C,4190A,14848,14848,826.0,4821.80,...,0.0,0.0,OUI,0.417773,ACTIF,CO,NaN,100,0.9000,95
3,A00104001402563,611.0,206122172,A00104,C,4190A,14861,14861,908.0,9460.74,...,0.0,0.0,OUI,0.508437,ACTIF,CO,NaN,100,0.9000,95
4,A00104716130632,603.0,35070,A00104,S,4190A,4018,14707,407.0,1530.09,...,NaN,NaN,OUI,0.615430,ACTIF,CO,NaN,91,0.8065,50


In [81]:
print(f'Nb lignes : {len(df)}')
print(f'Nb champs : {len(df.columns)}')

Nb lignes : 90247
Nb champs : 58


In [82]:
print(df.CONTRAT.value_counts())
print(df.CONTRAT.value_counts()/df.shape[0])

CONTRAT
ACTIF    80117
RESIL    10130
Name: count, dtype: int64
CONTRAT
ACTIF    0.887753
RESIL    0.112247
Name: count, dtype: float64


📌 POINT : 
----------
- Notre dataframe contient 90247 et 58 colonnes. On peut directement avoir l'intuition qu'il faudra surement réaliser une sélection de variable afin de rendre notre modèle plus performants et eviter la multicolinéarité.
- La variable que nous cherchons à predire est la variable "CONTRAT". Cette dernière nous indique si un client est toujours assuré ou non. On remarque directement que notre cible est désequilibrée, avec majoritairement (90%) des clients actifs contre une minorité de clients résiliés (10%).


### Afin de rendre l'inetrpretation des résultats plus simple nous avons décidé de modifier le noms de variables de bases qui n'était pas forcement très parlant : 

In [83]:
df.columns

Index(['NO_AFR', 'CDUSGAUT', 'IDECON', 'CD_AGT', 'CD_FML', 'CDPRGES',
       'DTDBUCON', 'DTEFTMVT', 'MMJECHPP', 'MTPAATTC', 'AUTO4R', 'NOCLIGES',
       'NUMFOY', 'CDMARVEH', 'DTOBTPDC', 'DTPMRMCI', 'LBMDLVH', 'NOTAREFF',
       'PUI_TRE', 'RN_VL_VH', 'S_2_N', 'S_3_N', 'S_1_N', 'S_0_N', 'S_3_O',
       'S_2_O', 'S_1_O', 'S_0_O', 'NIVBM', 'MTPAAREF', 'CDMCE', 'NBCTRES',
       'ANCCLI', 'CD_CSP', 'CD_SEX', 'DT_NAI', 'CDSITFAM', 'DEPT', 'REGION',
       'NBCTACT', 'AU4R', 'MH', 'DI', 'SA', 'IV', 'RESAU4R', 'RESMH', 'RESSA',
       'RESIV', 'RESDI', 'CLIACTIF', 'U', 'CONTRAT', 'ETAT', 'MOTIFRSL',
       'COEFCOMM', 'COEFPFLT', 'CRM'],
      dtype='object')

In [84]:
renommage_vars = {
    "NO_AFR": "numero_affaire",
    "CDUSGAUT": "code_usage_auto",
    "IDECON": "id_contrat",
    "CD_AGT": "code_agent",
    "CD_FML": "code_formule",
    "CDPRGES": "code_produit_gestion",
    "DTDBUCON": "date_debut_contrat",
    "DTEFTMVT": "date_dernier_mouvement",
    "MMJECHPP": "echeance_contrat_mmjj",
    "MTPAATTC": "prime_annuelle_ttc",
    "AUTO4R": "auto_4_roues",
    "NOCLIGES": "numero_client_gestion",
    "NUMFOY": "numero_foyer",
    "CDMARVEH": "code_marque_vehicule",
    "DTOBTPDC": "date_obtention_permis",
    "DTPMRMCI": "date_mise_circulation",
    "LBMDLVH": "libelle_modele_vehicule",
    "NOTAREFF": "numero_tarif",
    "PUI_TRE": "puissance_fiscale",
    "RN_VL_VH": "rang_valeur_vehicule",
    "S_2_N": "nb_sinistres_2_ans_non_resp",
    "S_3_N": "nb_sinistres_3_ans_non_resp",
    "S_1_N": "nb_sinistres_1_an_non_resp",
    "S_0_N": "nb_sinistres_0_an_non_resp",
    "S_3_O": "nb_sinistres_3_ans_resp",
    "S_2_O": "nb_sinistres_2_ans_resp",
    "S_1_O": "nb_sinistres_1_an_resp",
    "S_0_O": "nb_sinistres_0_an_resp",
    "NIVBM": "niveau_bonus_malus",
    "MTPAAREF": "prime_reference",
    "CDMCE": "code_marche",
    "NBCTRES": "nb_contrats_resilies",
    "ANCCLI": "anciennete_client",
    "CD_CSP": "code_csp",
    "CD_SEX": "sexe",
    "DT_NAI": "date_naissance",
    "CDSITFAM": "situation_familiale",
    "DEPT": "departement",
    "REGION": "region",
    "NBCTACT": "nb_contrats_actifs",
    "AU4R": "nb_contrats_actifs_auto",
    "MH": "nb_contrats_actifs_habitation",
    "DI": "nb_contrats_actifs_divers",
    "SA": "nb_contrats_actifs_sante",
    "IV": "nb_contrats_actifs_vie",
    "RESAU4R": "nb_contrats_auto_resilies",
    "RESMH": "nb_contrats_habitation_resilies",
    "RESSA": "nb_contrats_sante_resilies",
    "RESIV": "nb_contrats_vie_resilies",
    "RESDI": "nb_contrats_divers_resilies",
    "CLIACTIF": "client_actif",
    "U": "score_u",
    "CONTRAT": "contrat",
    "ETAT": "etat",
    "MOTIFRSL": "motif_resiliation",
    "COEFCOMM": "coefficient_commission",
    "COEFPFLT": "coefficient_profil_tarif",
    "CRM": "crm"
}


In [85]:
df.rename(columns=renommage_vars, inplace=True)

# 2. Compréhension des données

### 2.1 Vérification du type des variables

In [86]:
df.info() # nous donne le type de stockage de chaque variable

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90247 entries, 0 to 90246
Data columns (total 58 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   numero_affaire                   90247 non-null  object 
 1   code_usage_auto                  90231 non-null  float64
 2   id_contrat                       90247 non-null  int64  
 3   code_agent                       90247 non-null  object 
 4   code_formule                     89702 non-null  object 
 5   code_produit_gestion             90247 non-null  object 
 6   date_debut_contrat               90247 non-null  int64  
 7   date_dernier_mouvement           90247 non-null  int64  
 8   echeance_contrat_mmjj            90231 non-null  float64
 9   prime_annuelle_ttc               90247 non-null  float64
 10  auto_4_roues                     90247 non-null  object 
 11  numero_client_gestion            90247 non-null  int64  
 12  numero_foyer      

On constate que beaucoup de nos variables ne semblent pas au bon type. Pour faciliter nos future interpretations il est preferable de corriger ce problème des maintenant :

#### 2.1.1 Type variables de temps : 

In [87]:
var_date = ['date_debut_contrat', 'date_dernier_mouvement', 'date_obtention_permis', 'date_mise_circulation', 'date_naissance']

In [88]:
df[var_date]

,date_debut_contrat,date_dernier_mouvement,date_obtention_permis,date_mise_circulation,date_naissance
0,14630,14812,10273.0,10683.0,3639.0
1,14701,14701,12877.0,11262.0,6093.0
2,14848,14848,14196.0,9678.0,-8391.0
3,14861,14861,14629.0,11870.0,7904.0
4,4018,14707,NaN,5935.0,-3633.0
...,...,...,...,...,...
90242,13340,14801,11197.0,11323.0,-66.0
90243,13504,14051,9212.0,10624.0,2181.0
90244,14335,14335,9303.0,10897.0,2702.0
90245,14474,14840,13559.0,10593.0,6627.0


On constate que nous sommes sur un format "non standard" pour des dates ! Ce ne sont pas des yyyymmdd, mais des jours depuis une origine (on eut supposer Excel). 

Les valeurs comme :

- 14812, 14630 pour date_debut_contrat → ressemblent à des jours depuis 1900 (comme dans Excel).

- -8391.0 pour date_naissance → indique que certaines dates sont avant la date d'origine.

Donc on est très probablement sur une date d'origine de 1900.

Mais il faut être vigilant car : Excel considère 1900 comme une année bissextile (ce qui est faux), donc il ajoute un jour fantôme.

Pandas ne fait pas cette erreur.

In [89]:
origin = pd.Timestamp("1900-01-01")
min_days = -40000   # environ l’an 1790 — on peut ajuster
max_days = 100000   # environ 2173

for col in var_date:
    # Ne garde que les valeurs valides dans la plage
    valid = df[col].between(min_days, max_days)
    
    # Convertit uniquement les valeurs valides, le reste devient NaT
    df[col] = pd.to_timedelta(np.where(valid, df[col], np.nan), unit="D") + origin

In [90]:
df[var_date]

,date_debut_contrat,date_dernier_mouvement,date_obtention_permis,date_mise_circulation,date_naissance
0,1940-01-22,1940-07-22,1928-02-17,1929-04-02,1909-12-19
1,1940-04-02,1940-04-02,1935-04-05,1930-11-02,1916-09-07
2,1940-08-27,1940-08-27,1938-11-14,1926-07-02,1877-01-10
3,1940-09-09,1940-09-09,1940-01-21,1932-07-02,1921-08-23
4,1911-01-02,1940-04-08,NaT,1916-04-02,1890-01-20
...,...,...,...,...,...
90242,1936-07-11,1940-07-11,1930-08-29,1931-01-02,1899-10-27
90243,1936-12-22,1938-06-22,1925-03-23,1929-02-02,1905-12-22
90244,1939-04-02,1939-04-02,1925-06-22,1929-11-02,1907-05-27
90245,1939-08-19,1940-08-19,1937-02-15,1929-01-02,1918-02-23


#### 2.1.2 Types variables catégorielles : 

In [91]:
var_cat = ["numero_affaire", "code_usage_auto", "id_contrat", "code_agent", "code_formule", "code_produit_gestion", "numero_client_gestion", "numero_foyer", "code_marque_vehicule", "libelle_modele_vehicule", "numero_tarif", "rang_valeur_vehicule", "code_marche", "code_csp", "sexe", "situation_familiale", "departement", "region", "client_actif", "contrat", "etat", "motif_resiliation", "crm"]

In [92]:
df[var_cat] = df[var_cat].astype("category")

#### 2.1.2 Types variables catégorielles : 

In [94]:
var_int = ["nb_sinistres_2_ans_non_resp", "nb_sinistres_3_ans_non_resp", "nb_sinistres_1_an_non_resp", "nb_sinistres_0_an_non_resp", "nb_sinistres_3_ans_resp", "nb_sinistres_2_ans_resp", "nb_sinistres_1_an_resp", "nb_sinistres_0_an_resp", "prime_reference", "nb_contrats_resilies", "nb_contrats_actifs", "nb_contrats_actifs_auto", "nb_contrats_actifs_habitation", "nb_contrats_actifs_divers", "nb_contrats_actifs_sante", "nb_contrats_actifs_vie", "nb_contrats_auto_resilies", "nb_contrats_habitation_resilies", "nb_contrats_sante_resilies", "nb_contrats_vie_resilies", "nb_contrats_divers_resilies"]

In [95]:
df[var_int] = df[var_int].astype("int64")

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [96]:
var_float = ["prime_annuelle_ttc", "puissance_fiscale"]

In [97]:
df[var_float] = df[var_float].astype("float64")

ValueError: could not convert string to float: 'D'

In [42]:
df.isnull().mean() 

numero_affaire                     0.000000
code_usage_auto                    0.000177
id_contrat                         0.000000
code_agent                         0.000000
code_formule                       0.006039
code_produit_gestion               0.000000
date_debut_contrat                 1.000000
date_dernier_mouvement             1.000000
echeance_contrat_mmjj              0.000177
prime_annuelle_ttc                 0.000000
auto_4_roues                       0.000000
numero_client_gestion              0.000000
numero_foyer                       0.000000
code_marque_vehicule               0.000964
date_obtention_permis              1.000000
date_mise_circulation              1.000000
libelle_modele_vehicule            0.001219
numero_tarif                       0.002959
puissance_fiscale                  0.001839
rang_valeur_vehicule               0.001884
nb_sinistres_2_ans_non_resp        0.894600
nb_sinistres_3_ans_non_resp        0.910280
nb_sinistres_1_an_non_resp      

### 2.2 Remplacement des modalités (bonus)

Afin de gagner en lisibilité, nous allons labelliser les modalités pour nos variables catégorielles

In [ ]:
dict_labels = {
    "Comptes": {
        "init_values": [0, 1, 2, 3, 4],
        "new_values": ['< 0 €', '[ 0 € ; 200 € [', '> 200 €', 'Sans']
    },
    "Historique_credit": {
        "init_values": [-1, 0, 1, 2, 3, 4],
        "new_values": [
            'Impayés passés',
            'Impayé en cours (autre banque)',
            'Jamais aucun crédit',
            'Crédits en cours sans retard',
            'Crédits en cours avec retard'
        ]
    },
    "Objet_credit": {
        "init_values": [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "new_values": [
            'Voiture neuve',
            "Voiture d'occasion",
            'Mobilier / Equipement',
            'Radio / Télévision',
            'Appareils ménagers',
            'Travaux',
            'Etude',
            'Vacances',
            'Formation',
            'Business',
            'Autres'
        ]
    },
    "Epargne": {
        "init_values": [0, 1, 2, 3, 4, 5],
        "new_values": [
            '< 100 €',
            '[ 100 € ; 500 € [',
            '[ 500 € ; 1_000 € [',
            '> 1000 €',
            'Inconnu / Pas de compte'
        ]
    },
    "Anciennete_emploi": {
        "init_values": [0, 1, 2, 3, 4, 5],
        "new_values": [
            'sans emploi',
            '< 1 an',
            '[ 1 an ; 4 ans [',
            '[ 4 ans ; 7 ans [',
            '> 7 ans'
        ]
    },
    "Situation_familiale": {
        "init_values": [0, 1, 2, 3, 4, 5],
        "new_values": [
            'H: divorcé/séparé',
            'F: divorcée / séparée',
            'H: célibataire',
            'H: marié / veuf',
            'F: célibataire'
        ]
    },
    "Garanties": {
        "init_values": [0, 1, 2, 3],
        "new_values": ['Pas de garantie', 'Co-emprunteur', 'Garant']
    },
    "Biens": {
        "init_values": [0, 1, 2, 3, 4],
        "new_values": ['Immobilier', 'Assurance-vie', 'Voiture ou autre', 'Aucun bien connu']
    },
    "Autres_credits": {
        "init_values": [0, 1, 2, 3],
        "new_values": ['Banque', 'Etablissement de crédit', 'Aucun']
    },
    "Statut_domicile": {
        "init_values": [0, 1, 2, 3],
        "new_values": ['Locataire', 'Propriétaire', 'Hébergement gratuit']
    },
    "Type_emploi": {
        "init_values": [0, 1, 2, 3, 4],
        "new_values": [
            'Chômeur / Non-qualifié - Non-résident',
            'Non-qualifié - résident',
            'Employé qualifié / Fonctionnaire',
            'Cadre / Indépendant / Employé hautement qualifié / Dirigeant'
        ]
    },
    "Telephone": {
        "init_values": [0, 1, 2],
        "new_values": ['Néant', 'Oui, enregistré sous le nom du client']
    },
    "Etranger": {
        "init_values": [0, 1, 2],
        "new_values": ['Oui', 'Non']
    }
}

In [ ]:
for field_name in dict_labels.keys():
    df = rename_field_categories(dataframe=df,field=field_name,init_values=dict_labels[field_name]["init_values"],new_values=dict_labels[field_name]["new_values"],replace=True)